## Leverage*
Leverage is a tool used for multiplying out returns. It can also lead to an increased loss. If we have a confident strategy, it is an effective way of increasing our returns. Note that leverage can come in many forms. The type we will be dealing with in this section will be on margin trading. This involves taking out a loan from your broker and adding it to your capital base in order to increase the returns of your trading strategy. Obviously in order to make this profitable, our returns should be greater than the interest needed to pay the debt.

The leverage of a portfolio is measured by the *leverage ratio*. This is given by
$$
\text{Leverage ratio} = \frac{\text{Debt}+\text{Capital base}}{\text{Capital base}}\,.
$$
Let us see how this works using a capital base of 100k, 50k debt, 2% interest and an assumed aggregate portfolio performance of 5%.

In [ ]:
capital_base = 100000.0
debt = 50000.0
i = 0.02
r_p = 0.05

r_no_lvg = capital_base * r_p
print 'Portfolio returns without leverage: {0}'.format(r_no_lvg)

int_pmt = i * debt
r_lvg = (capital_base + debt) * r_p
r_total = r_lvg - int_pmt
r_pct_lvg = r_total / capital_base
lvg_ratio = (capital_base + debt) / capital_base
print 'Portfolio returns with leverage and interest: {0}'.format(r_total)
print 'Percentage returns with {1}x leverage and {2}% interest: {0}'.format(r_pct_lvg, lvg_ratio, i * 100)

Comparing returns of different investment opportunities without taking risk into account is meaningless. We want to see whether a (high) return stream is due to a (high) amount of risks taken rather than a well-performing strategy. Thus, we have risk-adjusted returns and methods of risk-adjustment. If we adjust several return streams for risk then we can consider them on equal footing, independent of the risk. This allows us to effectively compare and determine which return streams are the best for a given desired risk profile.

The (ex-ante) Sharpe ratio is a measure for calculating risk-adjusted return, given by $$ S_a=\frac{\mathbb{E}[R_a-R_b]}{\sigma_a}=\frac{\mathbb{E}[R_a-R_b]}{\sqrt{\text{var}(R_a-R_b)}}\,, $$ where $R_a$ is the portfolio return of the asset, $R_b$ the return of a benchmark asset (e.g. risk-free rate of return, or the return of an index such as the HSI), and $\sigma_a$ is the standard deviation of the asset excess return.

A higher Sharpe Ratio indicates that you are getting more return relative to the risk that your strategy is taking on. We compare the Sharpe Ratio of two strategies of interest and then pick the strategy with the better Sharpe Ratio. Using leverage, we can multiply the returns up to the desired level. Assuming constraints like capital capacity don't kick in, adding more money through leverage can bring a 2% per year strategy up to a 10% per year strategy while maintaining the same Sharpe. We can also invest a fraction of your available capital to bring a per year strategy's risk down to acceptable levels.

In [ ]:
# Note these are all expected returns. You need to validate that
# your strategy will continue to produce these returns and volatility using other methods.
# Just measuring historically is a very poor predictor of future performance.
# We'll for now assume you've already validated that you can expect to keep seeing numbers similar to these.

strat_A_ann_return = 0.22
strat_A_ann_vol = 0.15

strat_B_ann_return = 0.05
strat_B_ann_vol = 0.02

# We'll assume a risk free rate of 0.02
risk_free_rate = 0.02

print 'Strategy A Sharpe: %s' % ((strat_A_ann_return - risk_free_rate) / strat_A_ann_vol)
print 'Strategy B Sharpe: %s' % ((strat_B_ann_return - risk_free_rate) / strat_B_ann_vol)

# Add in leverage to B

leverage = 3
# Expressed in returns

print 'Strategy B Sharpe: %s' % (
    (strat_B_ann_return * leverage - risk_free_rate * leverage) / (strat_B_ann_vol * leverage)
)
print 'Strategy B Levered Annual Returns: %s' % (
    (strat_B_ann_return * leverage - risk_free_rate * leverage)
)

## Instabilities*

We have seen how autocorrelation leads to instability of the sample covariance matrix. Statistical instability in general is present in any time-evolving behaviour that is coupled with uncertainty, and we should be aware of them.

Generally instability of estimates. When it comes to observing data to deduce information about a parameter in our model, this is an estimate. There is an uncertainty in this estimate, which we must account for using the mean and standard deviation.

As an example, let us look at the rolling Sharpe ratio of an asset to see how it changes through time.

In [ ]:
def sharpe_ratio(asset, riskfree):
    return np.mean(asset - riskfree)/np.std(asset - riskfree)

start = '2012-01-01'
end = '2015-01-01'
# Use an HSI ETF as our risk-free rate of return
hsi_ret = ut.get_pricing('2800.HK', src='yhoo', start_date=start, end_date=end, field='close').dropna().pct_change()[1:]
pricing = ut.get_pricing('0700.HK', src='yhoo', start_date=start, end_date=end, field='close').dropna()
returns = pricing.pct_change()[1:] # Get the returns on the asset

# Compute the running Sharpe ratio
running_sharpe = [sharpe_ratio(returns[i-90:i], hsi_ret[i-90:i]) for i in range(90, len(returns))]

# Compute the mean and std of the running Sharpe ratios up to 100 days before the end
mean_rs = np.mean(running_sharpe[:-100])
std_rs = np.std(running_sharpe[:-100])

_, ax1 = plt.subplots()
ax1.plot(range(90, len(returns)), running_sharpe)
ticks = ax1.get_xticks()
ax1.set_xticklabels([pricing.index[int(i)].date() for i in ticks[:-1]]) # Label x-axis with dates


# Plot its mean and the +/- 1 standard deviation lines
ax1.axhline(mean_rs)
ax1.axhline(mean_rs + std_rs, linestyle='--')
ax1.axhline(mean_rs - std_rs, linestyle='--')

# Indicate where we computed the mean and standard deviations
# Everything after this is 'out of sample' which we are comparing with the estimated mean and std
ax1.axvline(len(returns) - 100, color='pink');
plt.xlabel('Date')
plt.ylabel('Sharpe Ratio')
plt.legend(['Sharpe Ratio', 'Mean', '+/- 1 Standard Deviation'])

print 'Mean of running Sharpe ratio:', mean_rs
print 'std of running Sharpe ratio:', std_rs

The standard deviation in this case is about a quarter of the range, so this data is extremely volatile. Taking this into account when looking ahead gave a better prediction than just using the mean, although we still observed data more than one standard deviation away. We could also compute the rolling mean of the Sharpe ratio to try and follow trends; but in that case, too, we should keep in mind the standard deviation.

The second instability comes in regression analysis.

In [6]:
t, s, m = "2017-hi-2".split('-')
print t

2017


[2, 3]